In [54]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer # TF-IDF векторизация
from sklearn.metrics.pairwise import linear_kernel # Для вычисления косинусной близости

In [55]:
data = pd.read_csv("./data/netflix_titles.csv")

In [56]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7787 entries, 0 to 7786
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       7787 non-null   object
 1   type          7787 non-null   object
 2   title         7787 non-null   object
 3   director      5398 non-null   object
 4   cast          7069 non-null   object
 5   country       7280 non-null   object
 6   date_added    7777 non-null   object
 7   release_year  7787 non-null   int64 
 8   rating        7780 non-null   object
 9   duration      7787 non-null   object
 10  listed_in     7787 non-null   object
 11  description   7787 non-null   object
dtypes: int64(1), object(11)
memory usage: 730.2+ KB


• show_id -id фильма,

• tуре - его тип (фильм или сериал),

• title - название,

• director - режиссер,

. cast - актерский состав,

• country - страна,

• date_added - дата добавления,

• release year - год выхода на экраны,

• rating - рейтинг,

• duration - продолжительность,

• listened_in -жанр(-ы),

• description - описание.

In [57]:
data.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, Rodolfo Valente, Vaneza Oliveira, Rafael Lozano, Viviane Porto, Mel Fronckowiak, Sergio Mamberti, Zezé Motta, Celso Frateschi",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi & Fantasy","In a future where the elite inhabit an island paradise far from the crowded slums, you get one chance to join the 3% saved from squalor."
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, Azalia Ortiz, Octavio Michel, Carmen Beato",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies","After a devastating earthquake hits Mexico City, trapped survivors from all walks of life wait to be rescued while trying desperately to stay alive."
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence Koh, Tommy Kuan, Josh Lai, Mark Lee, Susan Leong, Benjamin Lim",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow soldiers are forced to confront a terrifying secret that's haunting their jungle island training camp."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly, Christopher Plummer, Crispin Glover, Martin Landau, Fred Tatasciore, Alan Oppenheimer, Tom Kane",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi & Fantasy","In a postapocalyptic world, rag-doll robots hide in fear from dangerous machines out to exterminate them, until a brave newcomer joins the group."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aaron Yoo, Liza Lapira, Jacob Pitts, Laurence Fishburne, Jack McGee, Josh Gad, Sam Golzari, Helen Carey, Jack Gilpin",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-counting experts with the intent of swindling millions out of Las Vegas casinos by playing blackjack.


In [58]:
# Далее учтём стоп-слова, т. е. предлоги и другие служебные части речи, которые не несут содержательной информации, и с учётом этого
# определим нашу модель:

model = TfidfVectorizer(stop_words='english')


In [59]:
# Заполним пропуски пустыми строками:

data['description'] = data['description'].fillna('')


In [60]:
# Трансформируем наши описания в матрицу:

feature_matrix = model.fit_transform(data['description'])


In [61]:
# Задание 2.2

# Сколько столбцов в получившейся матрице?

feature_matrix.shape[1]


17905

In [62]:
# Теперь необходимо вычислить косинусную близость. Можно сделать это так:

cosine_sim = linear_kernel(feature_matrix, feature_matrix)


In [80]:
# Обратите внимание! Мы используем здесь linear kernel (), а не cosine_similarity(), так как в косинусном расстоянии в знаменателе
# реализуется нормировка векторов, а TF-IDF создаёт уже нормализованные векторы.
# Вернём индексацию и уберём дубликаты из данных:

indices = pd.Series(data.index, index=data['title']).drop_duplicates()


In [104]:
# Теперь пропишем функцию для создания рекомендаций:
def get_recommendations(title):
    idx = indices[title]
    # вычисляем попарные коэффициенты косинусной близости
    scores = list(enumerate(cosine_sim[idx]))
    # сортируем фильмы на основании коэффициентов косинусной близости по убыванию
    scores = sorted(scores, key=lambda x: x[1], reverse=True)
    # выбираем десять наибольших значений косинусной близости; нулевую не берём, т. к. это тот же фильм
    scores = scores[1:11]
    # забираем индексы
    ind_movie = [i[0] for i in scores]
    # возвращаем названия по индексам
    return data['title'].iloc[ind_movie]


In [106]:
# Задание 2.3

# Найдите вторую рекомендацию для детского фильма "Balto", вышедшего на экраны в 1995 году:

get_recommendations('Balto')


709                Balto 2: Wolf Quest
7446                           Vroomiz
1338    Chilling Adventures of Sabrina
7388                          Vampires
1770                          Dinotrux
2767                     Hold the Dark
5540                 Shanghai Fortress
4041                             Mercy
2582                       Half & Half
1365        Christmas in the Heartland
Name: title, dtype: object

---

In [110]:
from surprise import Dataset
from surprise import Reader
from surprise.dataset import BUILTIN_DATASETS #с помощью данного объекта мы можем использовать встроенные датасеты

data = Dataset.load_from_file(
    "./data/u.data.txt",
    reader=Reader(line_format="user item rating timestamp", sep="\t"),
)


In [114]:
df = pd.DataFrame(data.raw_ratings, columns=[
                  'userId', 'movieId', 'rating', 'timestamp'])


In [117]:
# Задание 3.1
# Сколько уникальных фильмов в наборе данных?
df.movieId.nunique()


1682

In [118]:
# Задание 3.2
# Сколько уникальных пользователей в наборе данных?
df['userId'].nunique()

943

In [119]:
# Задание 3.3
# Какая оценка встречается в наборе данных чаще всего? Введите ответ в виде целого числа.
df['rating'].value_counts()


4.0    34174
3.0    27145
5.0    21201
2.0    11370
1.0     6110
Name: rating, dtype: int64

In [123]:
# Задание 3.4
# Разбейте данные на обучающую и тестовую выборки. Объём тестовой выборки должен составлять 25 % от общего объёма данных. В качестве
# значения параметра random state возьмите число 13.
# Сколько объектов попало в тестовую выборку?
from surprise.model_selection import train_test_split

trainset, testset = train_test_split(data, test_size=0.25, random_state=13)
len(testset)


25000

In [ ]:
# Импортируем функции для построения рекомендательных систем (SVD - для model-based-подхода и KNNBasic - для memory-basic-подхода) и
# для оценки качества результата.
from surprise import SVD, KNNBasic, accuracy

# Теперь реализуем обычную коллаборативную фильтрацию. Выберем оценку схожести через косинусную близость и item-based-подход:
sim_options = {
    'name': 'cosine',
    'user_based': False
}
 
knn = KNNBasic(sim_options=sim_options)
